In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

%cd ../../src

/media/premium/common-biscuit/main/planogram_biscuit/src


In [2]:
import os
import sys
import json

In [3]:
from scoring.Locator import Locator
locator = Locator()

In [4]:
 
pred_dir = '/media/premium/common-biscuit/main/planogram_biscuit/data/output/image_annotations/integrated/op_annotations_rack'
predictions = os.listdir(pred_dir)

In [5]:
predictions[2]

'PHOTO-2021-07-21-07-45-43 2.json'

In [6]:
missisng_image = '/media/premium/common-biscuit/main/planogram_biscuit/data/output/image_annotations/integrated/op_annotations_rack/PHOTO-2021-07-22-08-29-32.json'

In [7]:
with open(os.path.join(pred_dir,missisng_image)) as file:
            data = json.loads(file.read())

In [8]:
rack_rows, packets,complete_rack = locator.get_rows_packets_predictions(data)
complete_rack

array([(138, 48, 712, 1391)], dtype=object)

In [9]:
rack_rows = locator.get_sorted_rows(rack_rows)

In [10]:
rack_rows

array([(173, 415, 691, 718), (197, 704, 698, 965), (165, 954, 703, 1355)],
      dtype=object)

In [11]:
avg_row_height = locator.get_average_height(rack_rows)
print(avg_row_height)

331.0


In [12]:
rack_rows = locator.find_missing_row_between(rack_rows, avg_row_height, 0.8, 2.2)
print(rack_rows)

[(173, 415, 691, 718) (197, 704, 698, 965) (165, 954, 703, 1355)]


In [13]:
import numpy as np
thresholds = {'missing_top_row_packet_overlap_thresh': 0.7, 'top_row_additional_pixels':25,
                'top_row_additional_thresh_pixels':50}
first_row = locator.find_missing_first_row_modified(rack_rows, avg_row_height, packets, complete_rack,
                                               thresholds['missing_top_row_packet_overlap_thresh'], 
                                               thresholds['top_row_additional_pixels'], 
                                               thresholds['top_row_additional_thresh_pixels'])
    
if first_row is not None:
    rack_rows = np.insert(rack_rows, 0, first_row)

first_row

(197, 59.0, 703, 415)

In [14]:
rack_rows

array([(197, 59.0, 703, 415), (173, 415, 691, 718), (197, 704, 698, 965),
       (165, 954, 703, 1355)], dtype=object)

In [22]:
first_row

In [33]:
packet_positions = locator.check_packet_row(rack_rows, packets, 0.7, 0.7)

In [36]:
len(packet_positions)

10

In [37]:
for i in packet_positions:
    print(len(i))

5
0
4
8
6
8
6
3
9
4


## Locate Packets Updated

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

%cd ../../src

/media/premium/common-biscuit/main/planogram_biscuit/src


In [2]:
pred_dir = '/media/premium/common-biscuit/main/planogram_biscuit/data/output/image_annotations/integrated/op_annotations_new/'
predictions = os.listdir('/media/premium/common-biscuit/main/planogram_biscuit/data/output/image_annotations/integrated/op_annotations_new')

In [3]:
from scoring.locate_packets_updated import *

In [4]:
location_dict = {}

rack_position_dict = {}
after_images_prediction = [pred for pred in predictions if "after" in pred]
prev_images_prediction = [pred for pred in predictions if "prev" in pred]

if len(after_images_prediction) and len(prev_images_prediction) > 1:
    for prediction in after_images_prediction:
        
        with open(os.path.join(pred_dir,prediction)) as file:
            data = json.loads(file.read())
        
        updated_packet_positions, rack_rows, rack_position_after = find_packet_positions(data)
        location_dict[prediction] = [np.array(updated_packet_positions), np.array(rack_rows)]
        rack_position_dict[prediction.replace('.json', '')] = rack_position_after

    for prediction in tqdm(prev_images_prediction):
        with open(os.path.join(pred_dir,prediction)) as file:
            data = json.loads(file.read())
        

        complete_rack_position = get_rack_position_from_after(prediction.replace('.json', ''), rack_position_dict)
        
        #print(rack_position_prev, len(data['complete_rack']))
        updated_packet_positions, rack_rows, rack_position = find_packet_positions_prev(data, complete_rack_position)
        location_dict[prediction] = [np.array(updated_packet_positions), np.array(rack_rows)]
else: 
    for prediction in predictions:
        with open(os.path.join(pred_dir,prediction)) as file:
            data = json.loads(file.read())
        
        updated_packet_positions, rack_rows = find_packet_positions(data)
        location_dict[prediction] = [np.array(updated_packet_positions), np.array(rack_rows)]

/home/sumit-tyagi/sumit-ssd/common-biscuit/installs/anaconda3/envs/pepsi/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/sumit-tyagi/sumit-ssd/common-biscuit/installs/anaconda3/envs/pepsi/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/media/premium/common-biscuit/main/planogram_biscuit/src/scoring/Locator.py:407: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  for i, row in enumerate(np.array(updated_packet_positions)):
/home/sumit-tyagi/sumit-ssd/common-biscuit/installs/anaconda3/envs/pepsi/lib/python3.6/site-packages/ipykernel_launcher.py:14: VisibleDeprecationWarning: Creating an ndarray 

MX-G06134_0100891375_70105295_19.12056000~-98.17375000_1_2021-11-10_prev.jpg None 0
MX-G06133_0101553772_80300988_19.12425000~-98.15720100_1_2021-11-11_prev.jpg None 1
MX-G06134_0100915320_70105295_19.12224100~-98.17061400_1_2021-11-10_prev.jpg None 0
MX-G06133_0102633685_80300988_19.11451600~-98.17777200_1_2021-11-11_prev.jpg None 1
MX-G06133_0103417063_80300988_19.12440100~-98.15488500_1_2021-11-11_prev.jpg None 1
MX-G06134_0100925703_70105295_19.12368000~-98.17147800_1_2021-11-10_prev.jpg None 1
MX-G06134_0101004807_70105295_19.12559000~-98.16517000_1_2021-11-10_prev.jpg None 0
MX-G06133_0103574094_80300988_19.12578400~-98.15399800_1_2021-11-11_prev.jpg None 1
MX-G06134_0100964020_70105295_19.11911000~-98.17079500_1_2021-11-10_prev.jpg None 0
MX-G06134_0100889498_70105295_19.11739300~-98.17419500_1_2021-11-10_prev.jpg None 1
MX-G06134_0100371273_70105295_19.11845000~-98.17544000_1_2021-11-10_prev.jpg None 1
MX-G06134_0100521406_70105295_19.12513000~-98.16984000_1_2021-11-10_prev.jpg

/media/premium/common-biscuit/main/planogram_biscuit/src/scoring/Locator.py:407: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  for i, row in enumerate(np.array(updated_packet_positions)):
/home/sumit-tyagi/sumit-ssd/common-biscuit/installs/anaconda3/envs/pepsi/lib/python3.6/site-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/media/premium/common-biscuit/main/planogram_biscuit/src/scoring/Locator.py:407: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tupl

MX-G06134_0100808202_70105295_19.11971000~-98.17147000_1_2021-11-10_prev.jpg None 1
MX-G06134_0101145801_70105295_19.11483000~-98.17874000_1_2021-11-10_prev.jpg None 1
MX-G06134_0100869244_70105295_19.12226500~-98.17065100_1_2021-11-10_prev.jpg None 0
MX-G06134_0100934847_70105295_19.12140000~-98.17754300_1_2021-11-11_prev.jpg None 0
MX-G06134_0101199650_70105295_19.11766500~-98.17423700_1_2021-11-10_prev.jpg None 1
MX-G06134_0102543779_70105295_19.12299160~-98.17208100_1_2021-11-10_prev.jpg None 0
MX-G06134_0101314864_70105295_19.12142000~-98.16903000_1_2021-11-11_prev.jpg None 0
MX-G06134_0102533088_70105295_19.11456870~-98.17955020_1_2021-11-10_prev.jpg None 0
MX-G06134_0101319740_70105295_19.12173000~-98.17541500_1_2021-11-10_prev.jpg None 1
MX-G06134_0101365384_70105295_19.12297600~-98.17260800_1_2021-11-10_prev.jpg None 1
MX-G06134_0101704373_70105295_19.11560000~-98.17850000_1_2021-11-11_prev.jpg None 1
MX-G06134_0101415020_70105295_19.12120000~-98.17267000_1_2021-11-10_prev.jpg

/media/premium/common-biscuit/main/planogram_biscuit/src/scoring/Locator.py:407: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  for i, row in enumerate(np.array(updated_packet_positions)):
/home/sumit-tyagi/sumit-ssd/common-biscuit/installs/anaconda3/envs/pepsi/lib/python3.6/site-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/sumit-tyagi/sumit-ssd/common-biscuit/installs/anaconda3/envs/pepsi/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/su

...

MX-G06225_0102645277_80203104_19.10139200~-98.16040000_1_2021-11-09_prev.jpg None 1
MX-G06225_0102703939_80203104_19.09583300~-98.15920800_1_2021-11-10_prev.jpg None 1
MX-G06225_0101628215_80203104_19.08938000~-98.16612000_1_2021-11-10_prev.jpg RIGHT 2
MX-G06225_0101910894_80203104_19.09532700~-98.16507000_1_2021-11-09_prev.jpg None 1
MX-G06225_0101643687_80203104_19.09782800~-98.16215100_1_2021-11-09_prev.jpg None 1
MX-G06225_0102716971_80203104_19.10078050~-98.16574860_1_2021-11-11_prev.jpg LEFT 2
MX-G06225_0101670885_80203104_19.09179000~-98.16509000_1_2021-11-11_prev.jpg None 1
MX-G06225_0102105818_80203104_19.09695400~-98.16022200_1_2021-11-11_prev.jpg None 1
MX-G06225_0102572331_80203104_19.10277940~-98.15957640_1_2021-11-09_prev.jpg None 1
MX-G06225_0101697090_80203104_19.09419000~-98.16405000_1_2021-11-09_prev.jpg None 1
MX-G06225_0101715516_80203104_19.10534000~-98.15610000_1_2021-11-09_prev.jpg None 1
MX-G06225_0101812319_80203104_19.09943000~-98.16174000_1_2021-11-09_prev.jp

 96%|█████████▋| 475/493 [00:06<00:00, 73.67it/s]/media/premium/common-biscuit/main/planogram_biscuit/src/scoring/Locator.py:407: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  for i, row in enumerate(np.array(updated_packet_positions)):
/home/sumit-tyagi/sumit-ssd/common-biscuit/installs/anaconda3/envs/pepsi/lib/python3.6/site-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/sumit-tyagi/sumit-ssd/common-biscuit/installs/anaconda3/envs/pepsi/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: M

MX-G06225_0102790610_80203104_19.10249600~-98.15855400_1_2021-11-09_prev.jpg None 1
MX-G06225_0102798201_80203104_19.10042570~-98.16854860_1_2021-11-11_prev.jpg None 1
MX-G06225_0103380586_80203104_19.12701066~-98.17548315_1_2021-11-11_prev.jpg None 1
MX-G06225_0102834169_80203104_19.10481500~-98.15663400_1_2021-11-09_prev.jpg None 1
MX-G06225_0103732516_80203104_19.05560173~-98.22157769_1_2021-11-10_prev.jpg None 0
MX-G06225_0102843702_80203104_19.10124020~-98.16225430_1_2021-11-11_prev.jpg None 1
MX-G06225_0103380649_80203104_19.09268807~-98.16459882_1_2021-11-11_prev.jpg None 0
MX-G06225_0103573702_80203104_19.08831000~-98.17697300_1_2021-11-11_prev.jpg None 1
MX-G06225_0103220606_80203104_19.10488900~-98.15757900_1_2021-11-09_prev.jpg None 1
MX-G06225_0103280257_80203104_19.09046600~-98.16538100_1_2021-11-10_prev.jpg LEFT 2
MX-G06225_0103427641_80203104_19.09907628~-98.16088396_1_2021-11-09_prev.jpg None 1
MX-G06225_0103292537_80203104_19.09365900~-98.16228100_1_2021-11-09_prev.jpg

100%|██████████| 493/493 [00:06<00:00, 75.56it/s]

MX-G06225_0103331395_80203104_19.10257700~-98.15991700_1_2021-11-09_prev.jpg None 1
MX-G06225_0103435818_80203104_19.10082181~-98.16559819_1_2021-11-11_prev.jpg None 1
MX-G06225_0103538111_80203104_19.09756337~-98.16252104_1_2021-11-09_prev.jpg RIGHT 2
MX-G06225_0103753751_80203104_19.09496282~-98.16082667_1_2021-11-10_prev.jpg None 1
MX-G06225_0103795223_80203104_19.09512197~-98.16387976_1_2021-11-09_prev.jpg None 1


In [7]:
len(prev_images_prediction)

493

In [7]:
print(rack_position_dict['MX-G06134_0101499899_70105295_19.11348700~-98.17738300_1_2021-11-11_after'])

None


In [6]:
def get_rack_position_from_after(prediction, position_dict_after):
    try:
        image_name = prediction.replace('prev','after')
        return position_dict_after[image_name]
    except:
        return None


p = 'MX-G06134_0101499899_70105295_19.11348700~-98.17738300_1_2021-11-11_prev.json'.replace('.json', '')

print(get_rack_position_from_after(p,rack_position_dict))

None


In [5]:
with open(os.path.join(pred_dir,'MX-G06134_0101499899_70105295_19.11348700~-98.17738300_1_2021-11-11_after.json')) as file:
            data = json.loads(file.read())
data

{'image_name': 'MX-G06134_0101499899_70105295_19.11348700~-98.17738300_1_2021-11-11_after.jpg',
 'packets': [{'sub_brand': 'cremax_de_nieve_vainilla',
   'x1': '863',
   'y1': '1677',
   'x2': '985',
   'y2': '1790'},
  {'sub_brand': 'delicias_bolsa',
   'x1': '26',
   'y1': '299',
   'x2': '180',
   'y2': '423'},
  {'sub_brand': 'emperador_chocolate',
   'x1': '2',
   'y1': '279',
   'x2': '88',
   'y2': '555'},
  {'sub_brand': 'mamut_clasico_paquetin',
   'x1': '832',
   'y1': '387',
   'x2': '918',
   'y2': '567'},
  {'sub_brand': 'cremax_de_nieve_chocolate',
   'x1': '1287',
   'y1': '32',
   'x2': '1480',
   'y2': '131'},
  {'sub_brand': 'marias_azucaradas_rollo',
   'x1': '1423',
   'y1': '411',
   'x2': '1483',
   'y2': '628'},
  {'sub_brand': 'emperador_piruetas',
   'x1': '702',
   'y1': '591',
   'x2': '781',
   'y2': '796'},
  {'sub_brand': 'cremax_de_nieve_chocolate',
   'x1': '1349',
   'y1': '412',
   'x2': '1460',
   'y2': '598'},
  {'sub_brand': 'chokis_clasico_rollo',


In [7]:
with open(os.path.join(pred_dir,'MX-G06134_0101499899_70105295_19.11348700~-98.17738300_1_2021-11-11_prev.json')) as file:
            data = json.loads(file.read())
data

{'image_name': 'MX-G06134_0101499899_70105295_19.11348700~-98.17738300_1_2021-11-11_prev.jpg',
 'packets': [{'sub_brand': 'crackets_regular_rollo',
   'x1': '941',
   'y1': '1091',
   'x2': '1010',
   'y2': '1295'},
  {'sub_brand': 'cremax_de_nieve_vainilla',
   'x1': '846',
   'y1': '1595',
   'x2': '933',
   'y2': '1678'},
  {'sub_brand': 'emperador_nocturno',
   'x1': '502',
   'y1': '605',
   'x2': '579',
   'y2': '783'},
  {'sub_brand': 'marias_clasica_rollo',
   'x1': '801',
   'y1': '401',
   'x2': '878',
   'y2': '569'},
  {'sub_brand': 'marias_azucaradas_rollo',
   'x1': '1308',
   'y1': '419',
   'x2': '1431',
   'y2': '639'},
  {'sub_brand': 'habaneras_integrales_rollo',
   'x1': '868',
   'y1': '400',
   'x2': '965',
   'y2': '554'},
  {'sub_brand': 'crackets_regular_rollo',
   'x1': '971',
   'y1': '1026',
   'x2': '1061',
   'y2': '1293'},
  {'sub_brand': 'chokis_chokomax_paquetin',
   'x1': '653',
   'y1': '1152',
   'x2': '730',
   'y2': '1321'},
  {'sub_brand': 'chokis

In [10]:
location_dict['MX-G06134_0101499899_70105295_19.11348700~-98.17738300_1_2021-11-11_prev.json']

[array([array([(0, 1, 63, 215), (52, 1, 248, 220), (232, 0, 425, 224),
        (413, 1, 490, 224), (477, 0, 560, 226), (545, 29, 631, 228),
        (690, 8, 834, 230), (819, 29, 972, 231)], dtype=object),
        array([(0.0, 284, 123.5, 461), (123.5, 284, 247.0, 461),
        (247.0, 284, 370.5, 461), (370.5, 284, 494.0, 461),
        (494.0, 284, 617.5, 461), (617.5, 284, 741.0, 461),
        (741.0, 284, 864.5, 461), (864.5, 284, 988.0, 461)], dtype=object),
        array([(2.0, 604.6153846153846, 68.0, 782.8461538461538),
        (68, 603, 158, 783), (134, 603, 232, 782), (195, 605, 287, 785),
        (260, 608, 347, 785), (330, 611, 407, 782), (397, 605, 463, 782),
        (454, 610, 518, 783), (502, 605, 579, 783), (552, 581, 646, 783),
        (618, 598, 708, 782), (672, 609, 769, 784), (723, 611, 824, 781),
        (777, 611, 884, 782),
        (884.0, 604.6153846153846, 939.5, 782.8461538461539),
        (939.5, 604.6153846153846, 995.0, 782.8461538461539)], dtype=object),
   

In [5]:
import os
file_location = '/media/premium/common-biscuit/main/planogram_biscuit/data/raw/compliance_output/'
os.path.join(file_location, 'MX-G06134_0100891375_70105295_19.12056000~-98.17375000_1_2021-11-10_prev.jpg'.split(".")[0]+'.jpg')

'/media/premium/common-biscuit/main/planogram_biscuit/data/raw/compliance_output/MX-G06134_0100891375_70105295_19.jpg'

In [5]:
for k,v in rack_position_dict.items():
    if k.split('.')[0] == "MX-G06134_0103819911_70105295_19":
        print(v)

RIGHT
